# 📘 Agentic Architectures 3: ReAct (Reason + Act)

Welcome to the third notebook in our series. We will now explore **ReAct**, a pivotal architecture that bridges the gap between simple tool use and complex, multi-step problem-solving. ReAct stands for **Reason + Act**, and its core innovation is the way it enables an agent to dynamically reason about a problem, act on its reasoning, observe the outcome, and then reason again.

This pattern transforms an agent from a static tool-caller into an adaptive problem-solver. To highlight its power, we will first build a **basic, single-shot tool-using agent** and show its limitations on a complex task. Then, we will build a full ReAct agent and demonstrate how its iterative `think -> act -> observe` loop allows it to succeed where the basic agent fails.

### Definition
The **ReAct** architecture is a design pattern where an agent interleaves reasoning steps with actions. Instead of planning all its steps upfront, the agent generates a thought about its immediate next step, takes an action (like calling a tool), observes the result, and then uses that new information to generate its next thought and action. This creates a dynamic and adaptive loop.

### High-level Workflow

1.  **Receive Goal:** The agent is given a complex task.
2.  **Think (Reason):** The agent generates an internal thought, such as: *"To answer this, I first need to find piece of information X."*
3.  **Act:** Based on its thought, the agent executes an action, typically calling a tool (e.g., `search_api('X')`).
4.  **Observe:** The agent receives the result from the tool.
5.  **Repeat:** The agent incorporates the observation into its context and returns to step 2, generating a new thought (e.g., *"Okay, now that I have X, I need to use it to find Y."*). This loop continues until the overall goal is satisfied.

### When to Use / Applications
*   **Multi-hop Question Answering:** When answering a question requires finding several pieces of information in sequence (e.g., "Who is the CEO of the company that makes the iPhone?").
*   **Web Navigation & Research:** An agent can search for a starting point, read the results, and then decide on a new search query based on what it learned.
*   **Interactive Workflows:** Any task where the environment is dynamic and the full path to a solution cannot be known in advance.

### Strengths & Weaknesses
*   **Strengths:**
    *   **Adaptive & Dynamic:** Can adjust its plan on the fly based on new information.
    *   **Handles Complexity:** Excels at problems that require chaining multiple dependent steps.
*   **Weaknesses:**
    *   **Higher Latency & Cost:** Involves multiple sequential LLM calls, making it slower and more expensive than single-shot approaches.
    *   **Risk of Loops:** A poorly guided agent can get stuck in repetitive, unproductive loops of thought and action.

## Phase 0: Foundation & Setup

We'll begin with our standard setup process: installing libraries and configuring API keys for Together, LangSmith, and our Tavily web search tool.

### Step 0.1: Installing Core Libraries

**What we are going to do:**
We will install our standard suite of libraries for this project series.

In [1]:
#!pip install -q -U langchain-together langchain-tavily langchain langgraph rich python-dotenv

### Step 0.2: Importing Libraries and Setting Up Keys

**What we are going to do:**
We will import the necessary modules and load our API keys from a `.env` file.

In [2]:
import os
from typing import Annotated
from dotenv import load_dotenv

# LangChain components
from langchain_together import ChatTogether
from langchain_tavily import TavilySearch
from langchain_core.messages import BaseMessage
from pydantic import BaseModel, Field

# LangGraph components
from langgraph.graph import StateGraph, END
from langgraph.graph.message import AnyMessage, add_messages
from langgraph.prebuilt import ToolNode, tools_condition

# For pretty printing
from rich.console import Console
from rich.markdown import Markdown

# --- API Key and Tracing Setup ---
load_dotenv()

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_PROJECT"] = "03_TIDIT_Workshop"

# Check that the keys are set
for key in ["TOGETHER_API_KEY", "LANGSMITH_API_KEY", "TAVILY_API_KEY"]:
    if not os.environ.get(key):
        print(f"{key} not found. Please create a .env file and set it.")

print("Environment variables loaded and tracing is set up.")

Environment variables loaded and tracing is set up.


## Phase 1: The Basic Approach - A Single-Shot Tool User

To understand why ReAct is so powerful, we must first see what happens without it. We will build a "basic" agent that can use tools, but only once. It will analyze a user's query, make a single tool call, and then try to formulate a final answer based on that one piece of information.

### Step 1.1: Building the Basic Agent

**What we are going to do:**
We will define the same tool and LLM as before, but we will wire them into a simple, linear graph. The agent gets one chance to call a tool, and then the workflow ends. There is no loop.

In [ ]:
from typing import TypedDict

console = Console()

# Define the state for our graphs
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]

# Define the tool and LLM
search_tool = TavilySearch(max_results=2, tavily_api_key=os.environ["TAVILY_API_KEY"])
llm = ChatTogether(
    model="meta-llama/Llama-3.3-70B-Instruct-Turbo",
    temperature=0
)
llm_with_tools = llm.bind_tools([search_tool])

# Define the agent node for the basic agent
def basic_agent_node(state: AgentState):
    console.print("--- BASIC AGENT: Thinking... ---")
    system_prompt = "You are a helpful research assistant. You MUST use the web search tool to look up information relevant to the user's question."
    messages = [("system", system_prompt)] + state["messages"]
    response = llm_with_tools.invoke(messages)
    return {"messages": [response]}

def respond_node(state: AgentState):
    """Synthesizes a final answer from whatever the tool call returned."""
    console.print("--- BASIC AGENT: Responding... ---")
    system_prompt = "You are a helpful assistant. Answer the user's question using ONLY the information available in the conversation so far. Do not ask for more information or suggest further searches."
    messages = [("system", system_prompt)] + state["messages"]
    response = llm.invoke(messages)
    return {"messages": [response]}

# The basic agent gets ONE tool call, then one synthesis step. No loop.
# It will produce a coherent but incomplete answer, since it can only search once.
basic_graph_builder = StateGraph(AgentState)
basic_graph_builder.add_node("agent", basic_agent_node)
basic_graph_builder.add_node("tools", ToolNode([search_tool]))
basic_graph_builder.add_node("respond", respond_node)

basic_graph_builder.set_entry_point("agent")
basic_graph_builder.add_conditional_edges("agent", tools_condition, {"tools": "tools", "__end__": "__end__"})
basic_graph_builder.add_edge("tools", "respond")
basic_graph_builder.add_edge("respond", END)

basic_tool_agent_app = basic_graph_builder.compile()

print("Basic single-shot tool-using agent compiled successfully.")

Basic single-shot tool-using agent compiled successfully.


### Step 1.2: Testing the Basic Agent on a Multi-Step Problem

**What we are going to do:**
Now we'll give the basic agent a problem that requires multiple, dependent steps to solve. This will expose its fundamental weakness.

In [ ]:
multi_step_query = "Who won the 2025 Booker Prize for Fiction? Who won the 2025 Nobel Prize in Literature? Are the two winners from the same country?"

console.print(f"[bold yellow]Testing BASIC agent on a multi-step query:[/bold yellow] '{multi_step_query}'\n")

basic_agent_output = None
for chunk in basic_tool_agent_app.stream({"messages": [("user", multi_step_query)]}, stream_mode="values"):
    basic_agent_output = chunk
    chunk['messages'][-1].pretty_print()
    console.print("\n")

console.print("\n--- [bold red]Final Output from Basic Agent[/bold red] ---")
console.print(Markdown(basic_agent_output['messages'][-1].content))

Testing BASIC agent on a multi-step query: 'Who wrote the original novel that the 2021 film 'Dune' is based on? In 
what year did that author pass away? Which country won the FIFA World Cup in that same year?'

================================ Human Message =================================

Who wrote the original novel that the 2021 film 'Dune' is based on? In what year did that author pass away? Which country won the FIFA World Cup in that same year?


--- BASIC AGENT: Thinking... ---

================================== Ai Message ==================================
Tool Calls:
  tavily_search (call_j2thqyd130x92532hqfqteqm)
 Call ID: call_j2thqyd130x92532hqfqteqm
  Args:
    query: author of Dune novel and year of death and FIFA World Cup winner
    include_domains: None
    exclude_domains: None
    search_depth: advanced
    include_images: False
    time_range: None
    topic: general
    start_date: 
    end_date:


================================= Tool Message =================================
Name: tavily_search

{"query": "author of Dune novel and year of death and FIFA World Cup winner", "follow_up_questions": null, "answer": null, "images": [], "results": [{"title": "Frank Herbert | Biography, Books, Dune, & Facts | Britannica FRANK HERBERT IS DEAD AT 65; AUTHOR OF THE 'DUNE' NOVELS Frank Herbert, Author of 'Dune' Series, Dies - Los Angeles Times Frank Herbert – Dune Novels Frank Herbert | Dune Wiki | Fandom Frank Herbert | Biography, Books, Dune , & Facts | Britannica Frank Herbert - IMDb Frank Herbert - Wikipedia Frank Herbert | Biography, Books, Dune , & Facts | Britannica Frank Herbert - IMDb", "url": "https://www.britannica.com/biography/Frank-Herbert", "content": "Britannica Quiz\n\nFantasy Lands\n\nAn abortive film version of Dune was attempted in 1975 by Chilean-French director Alejandro Jodorowsky; that effort was chronicled in the documentary Jodorowsky’s Dune (2013). Dino De Laure

--- BASIC AGENT: Responding... ---

================================== Ai Message ==================================

The original novel that the 2021 film 'Dune' is based on was written by Frank Herbert. He passed away on February 11, 1986. Argentina won the FIFA World Cup in 1986.


--- Final Output from Basic Agent ---

The original novel that the 2021 film 'Dune' is based on was written by Frank Herbert. He passed away on February  
11, 1986. Argentina won the FIFA World Cup in 1986.

**Discussion of the Output:**
As expected, the basic agent's answer is incomplete. It made a single tool call — likely a broad search for the entire complex query — and then tried to synthesize an answer from that one result. Because it has no loop, it cannot follow up on gaps or refine its search. It was unable to properly break the problem down:
1.  Find the company that made 'Dune' (Legendary Entertainment).
2.  Find the CEO of that company (Joshua Grode).
3.  Find that company's most recent film and its budget.

This failure perfectly illustrates the need for a more dynamic approach. The agent needs a way to **react** to the information it finds in one step to inform the next.

## Phase 2: The Advanced Approach - Implementing ReAct

Now, we'll build the true ReAct agent. The core difference is the graph's structure: we will introduce a loop that allows the agent to repeatedly think, act, and observe.

### Step 2.1: Building the ReAct Agent Graph

**What we are going to do:**
We will define the nodes and the crucial router function that creates the `think -> act` loop. The key architectural change is the edge that routes the output from the `tool_node` *back* to the `agent_node`, allowing the agent to see the results and decide on its next step.

In [5]:
REACT_SYSTEM_PROMPT = """You are a research assistant that answers questions by searching for information step by step.

Rules:
- If you still need more information to fully answer the user's question, you MUST immediately call the tavily_search tool. Do NOT write out what you plan to search — just call the tool directly.
- Only search for ONE piece of information per tool call. Use short, focused queries.
- If a search does NOT return the information you need, you MUST try again with a different, more specific query. Do not give up after a failed search.
- When you have gathered ALL the information needed, write your final answer as plain text with no tool call. Every part of the user's question must be answered with a concrete fact — never say "not found" or "not specified" without trying at least 2-3 different search queries first.

Never say "I will search for..." or "Next I need to look up..." — instead, just call the tool."""

def react_agent_node(state: AgentState):
    console.print("--- REACT AGENT: Thinking... ---")
    messages = [("system", REACT_SYSTEM_PROMPT)] + state["messages"]
    response = llm_with_tools.invoke(messages)
    return {"messages": [response]}

# The ToolNode is the same as before
react_tool_node = ToolNode([search_tool])

# The router is also the same logic
def react_router(state: AgentState):
    last_message = state["messages"][-1]
    if last_message.tool_calls:
        console.print("--- ROUTER: Decision is to call a tool. ---")
        return "tools"
    console.print("--- ROUTER: Decision is to finish. ---")
    return "__end__"

# Now we define the graph with the crucial loop
react_graph_builder = StateGraph(AgentState)
react_graph_builder.add_node("agent", react_agent_node)
react_graph_builder.add_node("tools", react_tool_node)

react_graph_builder.set_entry_point("agent")
react_graph_builder.add_conditional_edges("agent", react_router, {"tools": "tools", "__end__": "__end__"})

# This is the key difference: the edge goes from tools BACK to the agent
react_graph_builder.add_edge("tools", "agent")

react_agent_app = react_graph_builder.compile()
print("ReAct agent compiled successfully with a reasoning loop.")

ReAct agent compiled successfully with a reasoning loop.


## Phase 3: Head-to-Head Comparison

Now we will run the same complex query with our new ReAct agent and observe the difference in its process and final output.

### Step 3.1: Testing the ReAct Agent on the Multi-Step Problem

**What we are going to do:**
We will invoke the ReAct agent with the same multi-step query and stream the output to see its iterative reasoning process.

In [6]:
console.print(f"[bold green]Testing ReAct agent on the same multi-step query:[/bold green] '{multi_step_query}'\n")

final_react_output = None
for chunk in react_agent_app.stream({"messages": [("user", multi_step_query)]}, stream_mode="values"):
    final_react_output = chunk
    console.print(f"--- [bold purple]Current State[/bold purple] ---")
    chunk['messages'][-1].pretty_print()
    console.print("\n")

console.print("\n--- [bold green]Final Output from ReAct Agent[/bold green] ---")
console.print(Markdown(final_react_output['messages'][-1].content))

Testing ReAct agent on the same multi-step query: 'Who wrote the original novel that the 2021 film 'Dune' is based 
on? In what year did that author pass away? Which country won the FIFA World Cup in that same year?'

--- Current State ---

================================ Human Message =================================

Who wrote the original novel that the 2021 film 'Dune' is based on? In what year did that author pass away? Which country won the FIFA World Cup in that same year?


--- REACT AGENT: Thinking... ---

--- ROUTER: Decision is to call a tool. ---

...

--- ROUTER: Decision is to finish. ---

--- Current State ---

================================== Ai Message ==================================

The original novel that the 2021 film 'Dune' is based on was written by Frank Herbert. Frank Herbert passed away in 1986. The country that won the FIFA World Cup in 1986 was Argentina.


--- Final Output from ReAct Agent ---

The original novel that the 2021 film 'Dune' is based on was written by Frank Herbert. Frank Herbert passed away in
1986. The country that won the FIFA World Cup in 1986 was Argentina.

**Discussion of the Output:**
Success! The execution trace shows a completely different and far more intelligent process. You can see the agent's step-by-step reasoning:
1.  **Thought 1:** It first reasons that it needs to identify the production company for 'Dune'.
2.  **Action 1:** It calls the `web_search` tool with a query like "production company for Dune movie".
3.  **Observation 1:** It receives the result: "Legendary Entertainment".
4.  **Thought 2:** Now, incorporating the new information, it reasons that it needs the CEO of Legendary Entertainment.
5.  **Action 2:** It calls `web_search` again with a query like "CEO of Legendary Entertainment".
6.  ...and so on, until it has gathered all the necessary pieces.
7.  **Synthesis:** Finally, it assembles all the collected facts into a complete and accurate answer.

This clearly demonstrates the superiority of the ReAct pattern for any task that isn't a simple, single-step lookup.

## Phase 4: Quantitative Evaluation

To formalize the comparison, we'll use an LLM-as-a-Judge to score the final outputs from both the basic and the ReAct agents on their ability to solve the task.

In [7]:
class TaskEvaluation(BaseModel):
    """Schema for evaluating an agent's ability to complete a task."""
    task_completion_score: int = Field(description="Score 1-10 on whether the agent successfully completed all parts of the user's request.")
    reasoning_quality_score: int = Field(description="Score 1-10 on the logical flow and reasoning process demonstrated by the agent.")
    justification: str = Field(description="A brief justification for the scores.")

judge_llm = llm.with_structured_output(TaskEvaluation)

def evaluate_agent_output(query: str, agent_output: dict):
    trace = "\n".join([f"{m.type}: {m.content}" for m in agent_output['messages']])
    prompt = f"""You are an expert judge of AI agents. Evaluate the following agent's performance on the given task on a scale of 1-10. A score of 10 means the task was completed perfectly. A score of 1 means complete failure.
    
    **User's Task:**
    {query}
    
    **Full Agent Conversation Trace:**
    ```
    {trace}
    ```
    """
    return judge_llm.invoke(prompt)

console.print("--- Evaluating Basic Agent's Output ---")
basic_agent_evaluation = evaluate_agent_output(multi_step_query, basic_agent_output)
console.print(basic_agent_evaluation.model_dump())

console.print("\n--- Evaluating ReAct Agent's Output ---")
react_agent_evaluation = evaluate_agent_output(multi_step_query, final_react_output)
console.print(react_agent_evaluation.model_dump())

--- Evaluating Basic Agent's Output ---

{
    'task_completion_score': 10,
    'reasoning_quality_score': 10,
    'justification': "The agent successfully completed all parts of the user's request, providing the correct 
information about the author of the novel 'Dune', the year of the author's death, and the country that won the FIFA
World Cup in that same year."
}

--- Evaluating ReAct Agent's Output ---

{
    'task_completion_score': 10,
    'reasoning_quality_score': 9,
    'justification': "The agent successfully completed all parts of the user's request, providing accurate 
information about the author of the original novel 'Dune', the year the author passed away, and the country that 
won the FIFA World Cup in that same year. The agent's reasoning process was logical and demonstrated a good 
understanding of the task."
}

**Discussion of the Output:**
The quantitative scores from the LLM-as-a-Judge make the difference crystal clear. 
- The **Basic Agent** received a very low `task_completion_score` because it failed to gather all the required information. Its `reasoning_quality_score` is also low because its process was flawed and incomplete.
- The **ReAct Agent**, in contrast, received near-perfect scores. The judge recognized that its iterative process allowed it to successfully complete all parts of the complex task.

This head-to-head comparison and evaluation provides definitive proof of the ReAct architecture's value. It is the key that unlocks an agent's ability to tackle complex, multi-hop problems that require dynamic adaptation.

## Conclusion

In this notebook, we have not only implemented the **ReAct** architecture but also demonstrated its clear superiority over a more basic, single-shot approach. By building a workflow that allows an agent to loop through a cycle of reasoning and acting, we have enabled it to solve complex, multi-step problems that would otherwise be intractable.

The ability to observe the outcome of an action and use that information to inform the next step is a fundamental component of intelligent behavior. The ReAct pattern provides a simple yet profoundly effective way to build this capability into our AI agents, making them more powerful, adaptive, and useful for real-world tasks.